In [ ]:
import pyvista as pv
import numpy as np
import matplotlib.pyplot as mt
import vtk 
import vtkmodules 
import gzip


In [ ]:
XFILES1TO20 = [
    "TS21z_X_R2_000100",
    "TS21z_X_R2_001000",
    "TS21z_X_R2_002000",
    "TS21z_X_R2_003000",
    "TS21z_X_R2_004000",
    "TS21z_X_R2_005000",
    "TS21z_X_R2_006000",
    "TS21z_X_R2_007000",
    "TS21z_X_R2_008000",
    "TS21z_X_R2_009000",
    "TS21z_X_R2_010000",
    "TS21z_X_R2_011000",
    "TS21z_X_R2_012000",
    "TS21z_X_R2_013000",
    "TS21z_X_R2_014000",
    "TS21z_X_R2_015000",
    "TS21z_X_R2_016000",
    "TS21z_X_R2_017000",
    "TS21z_X_R2_018000",
    "TS21z_X_R2_019000",
    "TS21z_X_R2_020000"
            ]

In [ ]:
# loading zip file and creating a zip object
# for file in XFILES1TO20:
#     with gzip.open(f'.\\zipped\\{file}.gz', 'rb') as f:
#         file_content = f.read()

#     f = open(f".\\data\\{file}.bin", "wb")
#     f.write(file_content)
#     f.close()

In [ ]:
stuff = np.fromfile('.\\data\\TS21z_X_R2_008000.bin')
type(stuff)

In [ ]:
# Converts a binary file of data to Vtk for the specific data sets of the 2006 IEE.
# fileSource: String - Path to the file to convert
# fileDestination: String - Path to the file to write  
# Return: Void
def binaryToVtk(fileSource, fileDestination):
    # Reader to get the data from the file. This describes the on-disk file
    # format so that VTK can parse it correctly.
    reader = vtk.vtkImageReader()
    reader.SetFileName(fileSource)
    # There are three dimensions in the file (X, Y, and Z)
    # Note that this file only stores the value for the X
    # component of the velocity, but it does this over the
    # whole 3D volume.
    reader.SetFileDimensionality(3)

    # There is one scalar field stored, and it is in big-endian floats
    reader.SetDataScalarTypeToFloat()
    reader.SetDataByteOrderToBigEndian()
    reader.SetNumberOfScalarComponents(1)

    # The first and last index of data values in X, Y, and Z
    reader.SetDataExtent(0, 749, 0, 374, 0, 99)

    # Picking an origin at zero, for no good reason
    reader.SetDataOrigin(0, 0, 0)

    # The data samples are 800 meters apart in X,Y,Z
    reader.SetDataSpacing(800, 800, 800)
    reader.Update()

    # Check if the file exists
    # Check if the file format is supported
    # Get the output data object
    output = reader.GetOutput()
    # Print image properties
    dimensions = output.GetDimensions()
    data_type = output.GetScalarTypeAsString()
    scalar_range = output.GetScalarRange()
    # Flip the image over in Y to make the coordinate system match
    # that of the data computation. The VTK image reader assumes
    # pixel (0,0) is at the upper-left, which is left-handed.
    reslice1 = vtk.vtkImageFlip()
    reslice1.SetInputData(reader.GetOutput())
    reslice1.SetFilteredAxis(1)
    reslice1.Update()
    # Write the data to a Structured Points file
    writer = vtk.vtkStructuredPointsWriter()
    writer.SetInputData(reslice1.GetOutput())
    writer.SetFileName(fileDestination)
    writer.SetFileTypeToBinary()
    writer.Write()

In [ ]:
binaryToVtk("TS21z_X_R2_008000.bin", "FunctionX8000.vtk")
binaryToVtk("TS21z_Y_R2_008000.bin", "FunctionY8000.vtk")
binaryToVtk("TS21z_Z_R2_008000.bin", "FunctionZ8000.vtk")

In [ ]:
for item in XFILES1TO20:
    binaryToVtk(".\\data\\"+item+'.bin',".\\data\\"+item+".vtk")
